<a href="https://colab.research.google.com/github/JngMkk/stzone/blob/main/stzone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [ ]:
driver = webdriver.Chrome(r'C:\Users\JngMK\Desktop\2021KNUpython2/chromedriver')
url = 'https://strikes.zone/game/211115DSBKTW'
driver.get(url)
time.sleep(20)

In [ ]:
# 이닝별 버튼 클릭
driver.find_element_by_css_selector('#__layout > div > div > section > section > header > div.match__type > button.type__btn.type__btn--inning').click()

In [ ]:
def inning_1():
    inning = driver.find_elements_by_class_name('inning__view')[0]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]  # 4번째 그림 심판이 판정한 스트라이크 + 볼
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_1
    cd_1 = pd.DataFrame([sb, xcd, ycd]).T
    cd_1.columns = ['판정', 'x좌표', 'y좌표']
    del cd_1['판정']
    cd_1.index = [[num*0+1 for num in sb],[hi for hi in sb]]

def inning_2():
    inning = driver.find_elements_by_class_name('inning__view')[1]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_2
    cd_2 = pd.DataFrame([sb, xcd, ycd]).T
    cd_2.columns = ['판정', 'x좌표', 'y좌표']
    del cd_2['판정']
    cd_2.index = [[num*0+2 for num in sb],[hi for hi in sb]]

def inning_3():
    inning = driver.find_elements_by_class_name('inning__view')[2]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_3
    cd_3 = pd.DataFrame([sb, xcd, ycd]).T
    cd_3.columns = ['판정', 'x좌표', 'y좌표']
    del cd_3['판정']
    cd_3.index = [[num*0+3 for num in sb],[hi for hi in sb]]

def inning_4():
    inning = driver.find_elements_by_class_name('inning__view')[3]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_4
    cd_4 = pd.DataFrame([sb, xcd, ycd]).T
    cd_4.columns = ['판정', 'x좌표', 'y좌표']
    del cd_4['판정']
    cd_4.index = [[num*0+4 for num in sb],[hi for hi in sb]]

def inning_5():
    inning = driver.find_elements_by_class_name('inning__view')[4]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_5
    cd_5 = pd.DataFrame([sb, xcd, ycd]).T
    cd_5.columns = ['판정', 'x좌표', 'y좌표']
    del cd_5['판정']
    cd_5.index = [[num*0+5 for num in sb],[hi for hi in sb]]

def inning_6():
    inning = driver.find_elements_by_class_name('inning__view')[5]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_6
    cd_6 = pd.DataFrame([sb, xcd, ycd]).T
    cd_6.columns = ['판정', 'x좌표', 'y좌표']
    del cd_6['판정']
    cd_6.index = [[num*0+6 for num in sb],[hi for hi in sb]]

def inning_7():
    inning = driver.find_elements_by_class_name('inning__view')[6]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_7
    cd_7 = pd.DataFrame([sb, xcd, ycd]).T
    cd_7.columns = ['판정', 'x좌표', 'y좌표']
    del cd_7['판정']
    cd_7.index = [[num*0+7 for num in sb],[hi for hi in sb]]

def inning_8():
    inning = driver.find_elements_by_class_name('inning__view')[7]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_8
    cd_8 = pd.DataFrame([sb, xcd, ycd]).T
    cd_8.columns = ['판정', 'x좌표', 'y좌표']
    del cd_8['판정']
    cd_8.index = [[num*0+8 for num in sb],[hi for hi in sb]]

def inning_9():
    inning = driver.find_elements_by_class_name('inning__view')[8]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_9
    cd_9 = pd.DataFrame([sb, xcd, ycd]).T
    cd_9.columns = ['판정', 'x좌표', 'y좌표']
    del cd_9['판정']
    cd_9.index = [[num*0+9 for num in sb],[hi for hi in sb]]

def inning_10():
    inning = driver.find_elements_by_class_name('inning__view')[9]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_10
    cd_10 = pd.DataFrame([sb, xcd, ycd]).T
    cd_10.columns = ['판정', 'x좌표', 'y좌표']
    del cd_10['판정']
    cd_10.index = [[num*0+10 for num in sb],[hi for hi in sb]]

def inning_11():
    inning = driver.find_elements_by_class_name('inning__view')[10]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_11
    cd_11 = pd.DataFrame([sb, xcd, ycd]).T
    cd_11.columns = ['판정', 'x좌표', 'y좌표']
    del cd_11['판정']
    cd_11.index = [[num*0+11 for num in sb],[hi for hi in sb]]

def inning_12():
    inning = driver.find_elements_by_class_name('inning__view')[11]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_12
    cd_12 = pd.DataFrame([sb, xcd, ycd]).T
    cd_12.columns = ['판정', 'x좌표', 'y좌표']
    del cd_12['판정']
    cd_12.index = [[num*0+12 for num in sb],[hi for hi in sb]]

def inning_13():
    inning = driver.find_elements_by_class_name('inning__view')[12]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_13
    cd_13 = pd.DataFrame([sb, xcd, ycd]).T
    cd_13.columns = ['판정', 'x좌표', 'y좌표']
    del cd_13['판정']
    cd_13.index = [[num*0+13 for num in sb],[hi for hi in sb]]

def inning_14():
    inning = driver.find_elements_by_class_name('inning__view')[13]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_14
    cd_14 = pd.DataFrame([sb, xcd, ycd]).T
    cd_14.columns = ['판정', 'x좌표', 'y좌표']
    del cd_14['판정']
    cd_14.index = [[num*0+14 for num in sb],[hi for hi in sb]]

def inning_15():
    inning = driver.find_elements_by_class_name('inning__view')[14]
    stzone = inning.find_elements_by_class_name('strike__zone')[3]
    svg = stzone.find_element_by_tag_name('svg')
    xcd = []
    ycd = []
    sb = []
    append1 = xcd.append
    append2 = ycd.append
    append3 = sb.append
    for n in range(100):
        try :
            img = svg.find_elements_by_tag_name('image')[n]
            append1(img.get_attribute('x'))
            append2(img.get_attribute('y'))
            if img.get_attribute('xlink:href') == '/_nuxt/img/1c8f58b.png' :
                append3(1)
            else :
                append3(0)
        except IndexError :
            break
    global cd_15
    cd_15 = pd.DataFrame([sb, xcd, ycd]).T
    cd_15.columns = ['판정', 'x좌표', 'y좌표']
    del cd_15['판정']
    cd_15.index = [[num*0+15 for num in sb],[hi for hi in sb]]

In [ ]:
# 한 경기 모든 이닝 크롤링 + 엑셀 저장
def inning_crawl():
    for n in range(20):
        try:
            driver.find_elements_by_class_name('search__data')[n].click()
            time.sleep(3)
            html = driver.page_source
            time.sleep(3)
            soup = bs(html, 'html.parser')
            time.sleep(3)
            match = soup.select('#__layout > div > div > section > section > header > div.match__info > div.match__team > span')[0].text + ' ' + soup.select('#__layout > div > div > section > section > header > div.match__info > div.match__team > span')[1].text+ ' ' + soup.select('#__layout > div > div > section > section > header > div.match__info > div.match__team > span')[2].text
            date = soup.select_one('#__layout > div > div > section > section > header > div.match__info > div.match__date').text.strip().split('투')[0]
            referee = soup.select_one('#__layout > div > div > section > section > header > div.match__info > div.match__date>span').text.split('(')[1].split('심')[0].split(' ')[0]
            try :
                inning_1()
                time.sleep(3)
                inning_2()
                time.sleep(3)
                inning_3()
                time.sleep(3)
                inning_4()
                time.sleep(3)
                inning_5()
                time.sleep(3)
                inning_6()
                time.sleep(3)
                inning_7()
                time.sleep(3)
                inning_8()
                time.sleep(3)
                inning_9()
                time.sleep(3)
                inning_10()
                time.sleep(3)
                inning_11()
                time.sleep(3)
                inning_12()
                time.sleep(3)
                inning_13()
                time.sleep(3)
                inning_14()
                time.sleep(3)
                inning_15()
                time.sleep(3)
            except IndexError :
                pass
            if len(driver.find_elements_by_class_name('inning__view')) == 15:
                df = pd.concat([cd_1,cd_2,cd_3,cd_4,cd_5,cd_6,cd_7,cd_8,cd_9,cd_10,cd_11,cd_12,cd_13,cd_14,cd_15])
                df.index.names = [f'{date}_{match}_{referee}', '판정']
                df.to_excel(f'./cd/{date}_{match}_{referee}.xlsx')
                time.sleep(5)
            elif len(driver.find_elements_by_class_name('inning__view')) == 14:
                df = pd.concat([cd_1,cd_2,cd_3,cd_4,cd_5,cd_6,cd_7,cd_8,cd_9,cd_10,cd_11,cd_12,cd_13,cd_14])
                df.index.names = [f'{date}_{match}_{referee}', '판정']
                df.to_excel(f'./cd/{date}_{match}_{referee}.xlsx')
                time.sleep(5)
            elif len(driver.find_elements_by_class_name('inning__view')) == 13:
                df = pd.concat([cd_1,cd_2,cd_3,cd_4,cd_5,cd_6,cd_7,cd_8,cd_9,cd_10,cd_11,cd_12,cd_13])
                df.index.names = [f'{date}_{match}_{referee}', '판정']
                df.to_excel(f'./cd/{date}_{match}_{referee}.xlsx')
                time.sleep(5)
            elif len(driver.find_elements_by_class_name('inning__view')) == 12:
                df = pd.concat([cd_1,cd_2,cd_3,cd_4,cd_5,cd_6,cd_7,cd_8,cd_9,cd_10,cd_11,cd_12])
                df.index.names = [f'{date}_{match}_{referee}', '판정']
                df.to_excel(f'./cd/{date}_{match}_{referee}.xlsx')
                time.sleep(5)
            elif len(driver.find_elements_by_class_name('inning__view')) == 11:
                df = pd.concat([cd_1,cd_2,cd_3,cd_4,cd_5,cd_6,cd_7,cd_8,cd_9,cd_10,cd_11])
                df.index.names = [f'{date}_{match}_{referee}', '판정']
                df.to_excel(f'./cd/{date}_{match}_{referee}.xlsx')
                time.sleep(5)
            elif len(driver.find_elements_by_class_name('inning__view')) == 10:
                df = pd.concat([cd_1,cd_2,cd_3,cd_4,cd_5,cd_6,cd_7,cd_8,cd_9,cd_10])
                df.index.names = [f'{date}_{match}_{referee}', '판정']
                df.to_excel(f'./cd/{date}_{match}_{referee}.xlsx')
                time.sleep(5)
            elif len(driver.find_elements_by_class_name('inning__view')) == 9:
                df = pd.concat([cd_1,cd_2,cd_3,cd_4,cd_5,cd_6,cd_7,cd_8,cd_9])
                df.index.names = [f'{date}_{match}_{referee}', '판정']
                df.to_excel(f'./cd/{date}_{match}_{referee}.xlsx')
                time.sleep(5)
            else :
                pass
        except IndexError:
            break

In [ ]:
# 달력 넘기기
def first_cal(num):
    driver.find_element_by_class_name('strike__zone__wrapper').click()
    time.sleep(3)
    driver.find_element_by_class_name('mx-input-wrapper').click()
    time.sleep(3)
    for i in range(num):
        driver.find_element_by_class_name('mx-icon-last-month').click()
        time.sleep(3)
        if i == int(num) :
            break

In [ ]:
# 날짜 넘어갈 때마다 달력 넘기기
def last_cal(num):
    driver.find_element_by_class_name('mx-input-wrapper').click()
    time.sleep(3)
    for i in range(num):
        driver.find_element_by_class_name('mx-icon-last-month').click()
        time.sleep(3)
        if i == int(num) :
            break

In [ ]:
# 크롤링
def game_crawl(num):
    for i in range(42):
        calender = driver.find_element_by_class_name('mx-calendar-content')
        month = calender.find_element_by_class_name('mx-panel.mx-panel-date')
        day = month.find_elements_by_tag_name('td')[i]
        time.sleep(3)
        if day.get_attribute('class') == 'cell cur-month' :
            day.click()
            time.sleep(3)
            driver.find_element_by_class_name('mx-datepicker-btn.mx-datepicker-btn-confirm').click()
            time.sleep(10)
            inning_crawl()
            last_cal(num)
        elif day.get_attribute('class') == 'cell cur-month actived' :
            day.click()
            time.sleep(3)
            driver.find_element_by_class_name('mx-datepicker-btn.mx-datepicker-btn-confirm').click()
            time.sleep(10)
            inning_crawl()
            last_cal(num)
        else :
            continue

In [ ]:
# 2021.10 : 1 ~ 2021.04 : 7
# 2020.11 : 12 ~ 2020.05 : 18
# 2019.10 : 25 ~ 2019.03 : 32
# 2018.11 : 36 ~ 2018.03 : 44
# 2017.10 : 49 ~ 2017.03 : 56

def crawl(num):         # 2020.10 : crawl(1) ~ 2017.30 : crawl(56)
    first_cal(num)
    game_crawl(num)